In [1]:
import pandas as pd
import requests
from datetime import datetime
import pytz

In [72]:
# cities = ['Berlin', 'Hamburg', 'Frankfurt','Munich','Stuttgart','Cologne','Hannover']

In [2]:
import requests

url = "https://aerodatabox.p.rapidapi.com/airports/search/location/52.31/13.24/km/50/16"

querystring = {"withFlightInfoOnly":"true"}

headers = {
	"X-RapidAPI-Key": "10dacbb09amshd249a3747f1fa23p143465jsn96e481f4c35e",
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"searchBy":{"lat":52.31,"lon":13.24},"items":[{"icao":"EDDB","iata":"BER","name":"Berlin, Berlin Brandenburg","shortName":"Brandenburg","municipalityName":"Berlin","location":{"lat":52.35139,"lon":13.493889},"countryCode":"DE"}]}


In [3]:
response.json()

{'searchBy': {'lat': 52.31, 'lon': 13.24},
 'items': [{'icao': 'EDDB',
   'iata': 'BER',
   'name': 'Berlin, Berlin Brandenburg',
   'shortName': 'Brandenburg',
   'municipalityName': 'Berlin',
   'location': {'lat': 52.35139, 'lon': 13.493889},
   'countryCode': 'DE'}]}

In [4]:
airport_df=pd.json_normalize(response.json()['items'])

In [5]:
airport_df

,icao,iata,name,shortName,municipalityName,countryCode,location.lat,location.lon
0,EDDB,BER,"Berlin, Berlin Brandenburg",Brandenburg,Berlin,DE,52.35139,13.493889


In [6]:
airport_df.drop(columns=['iata', 'shortName','municipalityName'], inplace=True)

In [7]:
airport_df

,icao,name,countryCode,location.lat,location.lon
0,EDDB,"Berlin, Berlin Brandenburg",DE,52.35139,13.493889


In [8]:
def icao_airport_codes(latitudes, longitudes):

  #assert len(latitudes) == len(longitudes)
  
  list_for_df = []

  for i in range(len(latitudes)):

    url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{latitudes[i]}/{longitudes[i]}/km/70/5"

    querystring = {"withFlightInfoOnly":"true"}

    headers = {
      "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
      "X-RapidAPI-Key": "10dacbb09amshd249a3747f1fa23p143465jsn96e481f4c35e"
    }
    

    response = requests.request("GET", url, headers=headers, params=querystring)
    airport_json = response.json()
    
    airport_df = pd.json_normalize(airport_json['items'])
    
    airport_df.drop(columns=['iata', 'shortName','municipalityName'], inplace=True)
    airport_df.rename(columns={'location.lat': 'latitude',
                             'location.lon': 'longitude',
                              'name': 'airport_name',
                              'countryCode': 'country_code'},
                    inplace=True)

    list_for_df.append(airport_df)

  return pd.concat(list_for_df, ignore_index=True)

la = [52.3112, 53.33, 50.0638, 48.0815, 48.4639, 50.5611, 52.22]
lo = [13.2418, 10.00, 08.4056, 11.3430, 09.1048, 6.5710, 9.43]

In [10]:
airport_df = icao_airport_codes(la,lo)
airport_df

,icao,airport_name,country_code,latitude,longitude
0,EDDB,"Berlin, Berlin Brandenburg",DE,52.35139,13.493889
1,EDDH,Hamburg,DE,53.63040,9.988229
2,EDDF,Frankfurt-am-Main,DE,50.02640,8.543129
3,EDDM,Munich,DE,48.35380,11.786100
4,EDDS,Stuttgart,DE,48.68990,9.221960
5,EDDK,"Cologne, Cologne Bonn",DE,50.86590,7.142739
6,EDDV,"Hanover, Hannover",DE,52.46110,9.685079
